# Building Coordinated AI Agents with LangGraph

LangGraph provides a framework for building coordinated multi-agent systems with specialized agents working together through structured communication.

## Getting Started with LangGraph

### Environment Setup

First, you should setup your environment with the following packages:

In [ ]:
# Install required packages
# pip install langgraph langgraph-supervisor langchain langchain-core langchain-tavily langchain-openai python-dotenv

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_tavily import TavilySearch
from langgraph.prebuilt import create_react_agent

load_dotenv()

### Creating Agents in LangGraph

LangGraph makes it very easy to create your first agent. Let's walk through how to create a general purpose assistant with web search functionality in a few lines of code.

In [ ]:
# Initialize the search tool
web_search = TavilySearch(max_results=3)

# Create the agent using create_react_agent()
agent = create_react_agent(
    model=ChatOpenAI(model="gpt-4o"),
    tools=[web_search],
    prompt="You are a helpful assistant that can search the web for information and summarize the results to enhance your output.",
)

In [ ]:
# Invoke the agent with a question
response = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Find the open and close prices of Apple's stocks for June 1, 2025."
            }
        ]
    }
)

print(response["messages"][-1].content)

### Creating a Supervisor Multi-Agent System

A supervisor is a special agent that manages the workflow between multiple agents. It is responsible for:

- Routing the workflow between agents
- Managing the conversation history
- Ensuring the agents are working together to achieve the goal

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph_supervisor import create_supervisor

# Define domain-specific agents with their own tools and prompts
agent1 = create_react_agent(
    model=ChatOpenAI(model="gpt-4o"),
    tools=[web_search],
    prompt="You are a bull agent - find positive investment indicators",
)
agent2 = create_react_agent(
    model=ChatOpenAI(model="gpt-4o"), 
    tools=[web_search],
    prompt="You are a bear agent - find negative investment indicators",
)
agent3 = create_react_agent(
    model=ChatOpenAI(model="gpt-4o"),
    tools=[web_search], 
    prompt="You are a chairman - make final investment decisions",
)

# Create a memory checkpointer to persist conversation history for the supervisor
memory = MemorySaver()

supervisor = create_supervisor(
    model=ChatOpenAI(model="gpt-4o"),
    agents=[agent1, agent2, agent3],
    prompt="Detailed system prompt instructing the model how to route the workflow.",
).compile(checkpointer=memory)

In [ ]:
# Call the supervisor with a user query
response = supervisor.invoke({
    "messages": [
        {
            "role": "user",
            "content": "Should I invest in NVDA stock? I want to hear both bullish and bearish arguments."
        }
    ]
})

print(response["messages"][-1].content)